# Using data generation internally

In [17]:
import pandas as pd
from lsynth import compute_upsilon
MODEL='./modelHRS72p05.gz'
DATA="./HRSvar72.csv"
df_real = pd.read_csv(DATA, keep_default_na=False,index_col=0)#.sample(100)
#df_real.columns = ['x'+str(i) for i in df_real.columns]
df_real

,PA019,PA099,PA101,PB000,PB132,PB082,PC001,PC005,PC010,PC053,...,PX065_R,PMARST,PRVIW,PNHM,DEGREE,GENDER,HISPANIC,RACE,SCHLYRS,cognition
1,76,0,0,2,8,5,2,5,5,5,...,6,4,14,0,2,1,5,1,12,10
2,70,0,2,1,9,2,2,5,5,5,...,6,3,14,0,2,2,5,1,12,14
3,68,0,1,3,6,5,2,5,5,5,...,6,2,14,0,2,2,5,1,13,10
4,79,0,2,2,8,4,2,1,5,5,...,1,1,14,0,4,1,5,1,16,13
5,73,0,2,2,7,3,2,5,5,5,...,1,1,14,0,4,2,5,1,16,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9179,73,0,4,3,0,4,5,1,1,5,...,1,1,14,0,1,1,5,2,12,9
9180,67,0,4,2,0,1,2,1,1,5,...,1,1,14,0,2,2,5,2,13,6
9181,76,0,4,2,4,3,2,1,5,5,...,1,1,14,0,2,1,5,2,14,7
9182,66,0,0,4,0,5,4,1,5,5,...,6,2,14,1,0,2,5,2,11,3


In [18]:
from quasinet.qnet import load_qnet
model=load_qnet(MODEL)
feature_names=model.feature_names


In [19]:
df_real = df_real[feature_names].copy()

In [44]:
# Baseline independent-column generator
ups_baseline, syn_baseline = compute_upsilon(
    num=100,
    model_path=MODEL,
    generate=True,
    gen_algorithm="BASELINE",
    orig_df=df_real,
    n_workers=11,
)
#print("Baseline mean Upsilon:", ups_baseline.mean())

Loading model from ./modelHRS72p05.gz ...
Generating 100 samples via BASELINE (independent columns).
Computing Upsilon via model.average_fidelity ...


average_fidelity(threads=11): 100%|███████████| 100/100 [00:10<00:00,  9.19it/s]

Baseline mean Upsilon: nan


In [22]:
# LSM generator using qsample
ups_lsm, syn_lsm = compute_upsilon(
    num=100,
    model_path=MODEL,
    generate=True,
    gen_algorithm="LSM",
    orig_df=df_real,
    n_workers=11,
)
print("LSM mean Upsilon:", ups_lsm.mean())

Loading model from ./modelHRS72p05.gz ...
Generating 100 samples via LSM (qsample).


qsample(LSM, threads=11): 100%|███████████████| 100/100 [00:12<00:00,  7.96it/s]


Computing Upsilon via model.average_fidelity ...


average_fidelity(threads=11): 100%|███████████| 100/100 [00:10<00:00,  9.84it/s]

LSM mean Upsilon: 0.8167999047718196


In [14]:
# CTGAN generator (requires sdv installed)
ups_ctgan, syn_ctgan = compute_upsilon(
    num=10,
    model_path=MODEL,
    generate=True,
    gen_algorithm="CTGAN",
    orig_df=df_real,
    n_workers=11,
)
syn_ctgandf = pd.DataFrame(syn_ctgan,columns=feature_names )
print("CTGAN mean Upsilon:", ups_ctgan.mean())

Loading model from ./modelHRS72p1.gz ...
Generating 10 samples via CTGAN.
Computing Upsilon via model.average_fidelity ...


average_fidelity(threads=11): 100%|█████████████| 10/10 [00:00<00:00, 10.06it/s]

CTGAN mean Upsilon: 0.7957492941139683


# Use any externally generated synthetic data

In [50]:
symdf=syn_ctgan
#symdf=syn_baseline
#symdf=syn_lsm

dfx=pd.DataFrame(symdf,columns=feature_names )
df_external=dfx.copy().astype(float).round(0).astype(int)

In [51]:
ups_baseline2, syn_x = compute_upsilon(
    model_path=MODEL,
    generate=False,
    syndata=df_external,
    n_workers=11,
)
print("LSM mean Upsilon:", ups_baseline2.mean())

Loading model from ./modelHRS72p05.gz ...
Using external syndata DataFrame with 10 rows.
Computing Upsilon via model.average_fidelity ...


average_fidelity(threads=11): 100%|█████████████| 10/10 [00:00<00:00, 15.74it/s]

LSM mean Upsilon: 0.7830061098395759
